In [ ]:
import boto3
from sagemaker import get_execution_role
from sagemaker.transformer import Transformer
import sagemaker

# Définir le nom du modèle et du fichier S3 contenant les données de test
model_name = 'xgboost-titanic-model'  # Remplacez par le nom de votre modèle
input_data_s3_path = f"s3://{bucket_name}/processed/test_data.csv"
output_data_s3_path = f"s3://{bucket_name}/predictions/output"

# Initialiser la session SageMaker et le rôle d'exécution
role = get_execution_role()
sagemaker_session = sagemaker.Session()

# Charger le modèle XGBoost déjà formé
xgb_model = sagemaker.model.Model(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region="eu-west-3", version="1.2-1"),
    model_data=f"s3://{bucket_name}/output/model.tar.gz",  # Le chemin du modèle sauvegardé
    role=role,
    sagemaker_session=sagemaker_session
)

# Créer un objet Transformer pour le Batch Transform
transformer = xgb_model.transformer(
    instance_count=1,  # Le nombre d'instances pour effectuer les prédictions
    instance_type='ml.m5.large',  # Type d'instance à utiliser
    strategy='SingleRecord',  # Pour effectuer des prédictions ligne par ligne
    output_path=output_data_s3_path,  # Le chemin de sortie pour les résultats des prédictions
    input_filter='$[0,-1]',  # Inclure toutes les colonnes, sauf la dernière (cible)
    join_source='Input',  # Garder les colonnes de l'entrée intactes dans la sortie
    content_type='text/csv',  # Format d'entrée des données
    split_type='Line',  # Séparer chaque ligne comme un enregistrement
)

# Lancer le job Batch Transform
transformer.transform(
    data=input_data_s3_path,  # Données d'entrée pour les prédictions
    content_type='text/csv',
    split_type='Line',  # Une ligne à la fois
    input_filter='$[0,-1]',  # Inclure toutes les colonnes sauf la cible
    join_source='Input',  # Joindre les prédictions aux entrées originales
    output_filter='$[0,-1]',  # Inclure la première colonne et la dernière (la prédiction)
    wait=True  # Attendre la fin du job avant de poursuivre
)

# Vérifier l'emplacement des prédictions
print(f"Les prédictions sont disponibles ici : {output_data_s3_path}")


In [ ]:
# Télécharger les résultats des prédictions
s3.download_file(bucket_name, 'predictions/output/predictions.csv', 'predictions.csv')

# Charger les résultats des prédictions
predictions_df = pd.read_csv('predictions.csv')

# Supposons que la première colonne est la prédiction
predictions = predictions_df['predicted_label']

# Charger les vraies valeurs de l'ensemble de test
y_test = pd.read_csv('test_data.csv')['Survived']

# Calculer l'accuracy
accuracy = (predictions == y_test).mean()
print(f"Accuracy: {accuracy}")
